In [ ]:
# user_id = 'notbobbobby'

# GET PLAYLISTS
# playlists = get_user_playlists(user_id, [discover_database=True])

# GET TRACKS
# tracks = get_user_tracks(user_id, playlists)

# GET FEATURES
# features = get_user_unique_tracks_and_features(user_id, tracks)

# COMBINE INTO ONE DATAFRAME
# df = pd.merge(tracks, playlists, how='outer', on=['playlist_id', 'playlist_name'])
# df = pd.merge(df, features, how='outer', on=['track_id', 'track_name'])

# Note: indieair showed missing songs, which indicates the discrepancies between
#       total_tracks of the playlists and output number of tracks

import os
import pandas as pd
import altair as alt

ROOT = os.getcwd()
def get_data(db, user, time, kind='library'):
    if db == '_history':
        path = os.path.expanduser(os.path.join(ROOT, db, user, '{}_{}_{}.json'.format(user, time, kind)))
    else:
        path = os.path.expanduser(os.path.join(ROOT, db, user, time, '{}_{}_{}.json'.format(user, time, kind)))
    return pd.read_json(path, orient='records', lines=True)

library = get_data('_db', 'notbobbobby', '2018-10-31')
discover = get_data('_db', 'discover', '2018-10-31')

history_n = get_data('_history', 'notbobbobby', '2018', kind='history')
history_d = get_data('_history', 'deedanvy', '2018', kind='history')
history_c = get_data('_history', 'c.bochulak', '2018', kind='history')
history = pd.concat([history_n, history_d, history_c])

from datetime import timedelta
history['played_at'] = history['played_at'].apply(lambda t: t - timedelta(hours=6))

In [ ]:
from sklearn.decomposition import PCA
from sklearn import preprocessing

def analyze(playlist_name, included_columns):
    playlist = library[library.playlist_name.isin([playlist_name])]
    pca = PCA(n_components=len(included_columns))
    result = pca.fit_transform(playlist[columns].values)
    return playlist \
        .assign(pca_one=result[:,-1]) \
        .assign(pca_two=result[:,-2])

columns = ['danceability', 'energy', 'valence', 'instrumentalness']
analysis = pd.concat([analyze(name, columns) for name in ["Vexento n' Chill", 'Seltic']])

In [ ]:
from sklearn.manifold import TSNE

#TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
def analyze_tsne(playlist_name, included_columns):
    playlist = library[library.playlist_name.isin([playlist_name])]
    tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
    result = tsne.fit_transform(playlist[columns].values)
    return playlist \
        .assign(pca_one=result[:,0]) \
        .assign(pca_two=result[:,1])

columns = ['danceability', 'energy', 'valence', 'instrumentalness']
analysis = pd.concat([analyze(name, columns) for name in ["Vexento n' Chill", 'Seltic']])

In [ ]:
alt.Chart(analysis) \
    .mark_circle() \
    .encode(
        x='pca_one:Q',
        y='pca_two:Q',
        color='playlist_name:N')